In [3]:
import os

from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
%cd /content/gdrive/MyDrive/Colab Notebooks/sae

/content/gdrive/MyDrive/Colab Notebooks/sae


In [5]:
! ls

EleutherAI_SAE_Tests.ipynb  LICENSE  pyproject.toml  README.md	sae


In [6]:
! pip install einops
! pip install jaxtyping
! pip install datasets
! pip install simple_parsing
! pip install accelerate
! pip install multiprocessing
! pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency r

In [ ]:
! python -m sae

In [ ]:
! python -m sae EleutherAI/pythia-160m togethercomputer/RedPajama-Data-1T-Sample --load_in_8bit=True --attn_implementation=eager --dataset_percentage 5

config.json: 100% 569/569 [00:00<00:00, 4.58MB/s]
model.safetensors: 100% 375M/375M [00:03<00:00, 111MB/s]
Generating train split: 930514 examples [01:22, 11213.39 examples/s]
tokenizer_config.json: 100% 396/396 [00:00<00:00, 3.62MB/s]
tokenizer.json: 100% 2.11M/2.11M [00:00<00:00, 7.41MB/s]
special_tokens_map.json: 100% 99.0/99.0 [00:00<00:00, 984kB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map (num_proc=6): 100% 46526/46526 [04:53<00:00, 158.75 examples/s]
Training on 'togethercomputer/RedPajama-Data-1T-Sample' (split 'train')
Storing model weights in torch.float16
Training on layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Auto-selected LR: 1.63e-04
Using 8-bit Adam from bitsandbytes
Weights & Biases not installed, skipping logging.
Number of SAE parameters: 453_288_960
Number of model parameters: 162_322_944
Training:   0% 0/4654 [00:00<?, ?it/s]2024-06-20 04:57:51.475677: I tensorflow/core/util/port.cc:1

In [27]:
# For some reason I'm still getting the error even with the --attn_implementation="eager"
# oops i just didn't commit all the changes like an... idiot

with open('/content/gdrive/MyDrive/Colab Notebooks/sae/sae/__main__.py', 'r') as f:
    lines = f.readlines()
    for i in range(50, 90):
        print(lines[i])

    

    ## ADD

    dataset_percentage: float = field(default=10.0, help="Percentage of the dataset to use")

    """Percentage of the dataset to train on."""





def load_artifacts(args: RunConfig, rank: int) -> tuple[PreTrainedModel, Dataset]:

    if args.load_in_8bit:

        dtype = torch.float16

    elif torch.cuda.is_bf16_supported():

        dtype = torch.bfloat16

    else:

        dtype = "auto"



    model = AutoModelForCausalLM.from_pretrained(

        args.model,

        attn_implementation=args.attn_implementation,

        device_map={"": f"cuda:{rank}"},

        quantization_config=(

            BitsAndBytesConfig(load_in_8bit=args.load_in_8bit)

            if args.load_in_8bit

            else None

        ),

        torch_dtype=dtype,

        token=args.hf_token,

    )



    dataset_percentage = int(args.dataset_percentage)

    dataset = load_dataset(

        args.dataset,

        ## ADD

        split=f'{args.split}[:{args.dataset_percentage}%]'

In [ ]:
# ykw maybe i'll try it programmatically

# okay that seemed to work

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from sae import SaeConfig, SaeTrainer, TrainConfig
from sae.data import chunk_and_tokenize

MODEL = "EleutherAI/pythia-160m"
dataset = load_dataset(
    "togethercomputer/RedPajama-Data-1T-Sample",
    split="train",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL, attn_implementation="eager")
tokenized = chunk_and_tokenize(dataset, tokenizer)


gpt = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map={"": "cuda"},
    torch_dtype=torch.bfloat16,
)

cfg = TrainConfig(
    SaeConfig(gpt.config.hidden_size), batch_size=16
)
trainer = SaeTrainer(cfg, tokenized, gpt)

trainer.fit()